#### OBJETIVO:  Generar mapas diarios a partir de los modelos seleccionados por sitio, considerando las variables en cada caso

In [1]:
import rasterio
import numpy as np
import pandas as pd
import joblib
from datetime import datetime, timedelta
from pathlib import Path
print("Libreria ok")

Libreria ok


In [ ]:
# ---------------- CONFIG ----------------
estacion = "CH"
modelo = "1"
year = 2024

# Paths de los modelos e imagenes
ruta_modelo = f"D:/Josefina/Proyectos/Tesis/{estacion}/modelos_py/"
ruta_imagenes = f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/dataset_ejemplo/Prediccion_{year}/tiff/"
ruta_salida = f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/salidas/SalidasDiarias/modelo_py/"

Path(ruta_salida).mkdir(parents=True, exist_ok=True)

#Levantar modelos
rf_model = joblib.load(
    ruta_modelo + f"01-RF-CV10-M{modelo}-{estacion}_220126.joblib"
)

#Poner en el mismo orden de las variables
features = rf_model.feature_names_in_
print("Orden de variables:", features)

#Funcion solo para leer los rasters
def read_raster(path):
    with rasterio.open(path) as src:
        data = src.read(1)
        profile = src.profile
    return data, profile

#Setear las fechas para leer siempre la misma informacion diaria
fecha_inicio = datetime(2024, 1, 1)
fecha_fin    = datetime(2024, 1, 31)
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")


for fecha in fechas:

    print("Procesando:", fecha.date())

    fecha_str = fecha.strftime("%Y-%m-%d")
    fecha_merra = fecha.strftime("%Y%m%d")
    fecha_juliana = fecha.strftime("%Y%j")

    # ---------------- PATHS ----------------
    paths = {
        "AOD_055": ruta_imagenes + f"00_MAIAC/00_MAIAC_IDW/IDW-{fecha_juliana}-MAIAC_raster.tif",
        "ndvi": ruta_imagenes + "01_NDVI/2024001-NDVI_raster.tif",
        "BCSMASS_dia": ruta_imagenes + f"04_MERRA-2_dia/{fecha_merra}-BCSMASS_raster.tif",
        "DUSMASS_dia": ruta_imagenes + f"04_MERRA-2_dia/{fecha_merra}-DUSMASS_raster.tif",
        "SO2SMASS_dia": ruta_imagenes + f"04_MERRA-2_dia/{fecha_merra}-SO2SMASS_raster.tif",
        "SO4SMASS_dia": ruta_imagenes + f"04_MERRA-2_dia/{fecha_merra}-SO4SMASS_raster.tif",
        "SSSMASS_dia": ruta_imagenes + f"04_MERRA-2_dia/{fecha_merra}-SSSMASS_raster.tif",
        # "sp_mean": ruta_imagenes + f"05_ERA5/{fecha_str}-SP_raster.tif",
        "blh_mean": ruta_imagenes + f"05_ERA5/{fecha_str}-BLH_raster.tif",
        "d2m_mean": ruta_imagenes + f"05_ERA5/{fecha_str}-D2M_raster.tif",
        "t2m_mean": ruta_imagenes + f"05_ERA5/{fecha_str}-T2M_raster.tif",
        "v10_mean": ruta_imagenes + f"05_ERA5/{fecha_str}-V10_raster.tif",
        "u10_mean": ruta_imagenes + f"05_ERA5/{fecha_str}-U10_raster.tif",
        "tp_mean": ruta_imagenes + f"05_ERA5/{fecha_str}-TP_raster.tif",
        "dayWeek": ruta_imagenes + f"06_weekDay/{fecha_str}-weekDay_raster.tif",
        "DEM": ruta_imagenes + "03_DEM/DEM_raster.tif",
    }

    # ---------------- STACK ----------------
    arrays = []
    for var in features:
        data, profile = read_raster(paths[var])
        arrays.append(data)

    stack = np.stack(arrays, axis=-1)
    n_rows, n_cols, n_vars = stack.shape

    X = stack.reshape(-1, n_vars)
    df = pd.DataFrame(X, columns=features)

    # ---------------- PREDICCIÓN ----------------
    mask = ~np.any(np.isnan(X), axis=1)
    X_valid = df.loc[mask, features]

    pred = rf_model.predict(X_valid)

    pred_full = np.full(n_rows * n_cols, np.nan)
    pred_full[mask] = pred
    pred_raster = pred_full.reshape(n_rows, n_cols)

    # ---------------- GUARDAR TIFF ----------------
    profile.update(dtype=rasterio.float32, count=1, compress="lzw")

    out_path = ruta_salida + f"PM_{fecha_str}_RF.tif"

    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(pred_raster.astype(np.float32), 1)


Orden de variables: ['AOD_055' 'ndvi' 'BCSMASS_dia' 'DUSMASS_dia' 'SO2SMASS_dia'
 'SO4SMASS_dia' 'SSSMASS_dia' 'blh_mean' 'd2m_mean' 't2m_mean' 'v10_mean'
 'u10_mean' 'tp_mean' 'dayWeek' 'DEM']
Procesando: 2024-01-01
Procesando: 2024-01-02
Procesando: 2024-01-03
Procesando: 2024-01-04
Procesando: 2024-01-05
Procesando: 2024-01-06
Procesando: 2024-01-07
Procesando: 2024-01-08
Procesando: 2024-01-09
Procesando: 2024-01-10
Procesando: 2024-01-11
Procesando: 2024-01-12
Procesando: 2024-01-13
Procesando: 2024-01-14
Procesando: 2024-01-15
Procesando: 2024-01-16
Procesando: 2024-01-17
Procesando: 2024-01-18
Procesando: 2024-01-19
Procesando: 2024-01-20
Procesando: 2024-01-21
Procesando: 2024-01-22
Procesando: 2024-01-23
Procesando: 2024-01-24
Procesando: 2024-01-25
Procesando: 2024-01-26
Procesando: 2024-01-27
Procesando: 2024-01-28
Procesando: 2024-01-29
Procesando: 2024-01-30
Procesando: 2024-01-31
